#5. IMPORTING AND TIDYING PEOPLE DATA

##5.01 Initial work.

In [0]:
#Importing basic libraries:
import pandas as pd
import numpy as np
import timeit
import ujson
import os

In [0]:
#Click on the link, copy the authorization code and paste it on the box.
from google.colab import drive
drive.mount('/content/drive')

In [2]:
#Now we have access to our own Google Drive account:
!ls -lh "/content/drive/My Drive/"

total 3.2G
drwx------ 2 root root 4.0K Jul 14 07:08  20180714_Deep_Learning
-rw------- 1 root root  76K Oct 22 18:13  6885.json
drwx------ 2 root root 4.0K Jun 26  2016 'avast! Files (XT1068, UWbYzWPVmAmwlrlD)'
-rw------- 1 root root 2.4M Nov 18 16:45  Cast_Crew_ids.csv
drwx------ 2 root root 4.0K Jul 18 18:38 'Colab Notebooks'
-rw------- 1 root root  44K Feb  4  2018  contacts.vcf
-rw------- 1 root root    1 Apr  9  2018  facturaIVA.gsheet
-rw------- 1 root root  50K Nov  9 13:01  genres.xlsx
-rw------- 1 root root 1.5M Oct 24 17:34  IMDB_5000_movie.csv
-rw------- 1 root root  12K Nov  3 10:33  InflationConverter.xlsx
-rw------- 1 root root    1 Nov 10 10:21 'Mis cuentas.gsheet'
-rw------- 1 root root  93K Nov 18 16:38  Movies_ids.csv
-rw------- 1 root root 2.2M Nov 16 17:17  MoviesUS.csv
-rw------- 1 root root 1.5G Oct 12 15:09  movies.zip
-rw------- 1 root root    1 Nov 17 16:27 'Nuestras Cuentas.gsheet'
-rw------- 1 root root 1.8G Oct 27 13:06  people.zip
-rw------- 1 root root 6.4

In [0]:
# Install the PyDrive wrapper & import libraries. This only needs to be done once per notebook.
!pip install -U -q PyDrive
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

# Authenticate and create the PyDrive client. This only needs to be done once per notebook.
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

In [0]:
#Getting files from my Google Drive account:

#people data
file_id = '1UnpPTqVcghfQzj55RIoLm8pBH_Rjdl4f'
downloaded = drive.CreateFile({'id': file_id})
downloaded.GetContentFile("people.zip")

#people ids
file_id = '19oOa9qjMFnDv19DHmNTTJKIuGW8X5BkQ'
downloaded = drive.CreateFile({'id': file_id})
downloaded.GetContentFile("Cast_Crew_ids.csv")

In [7]:
!ls -lh

total 1.5G
-rw-r--r-- 1 root root 2.5K Nov 18 09:03 adc.json
-rw-r--r-- 1 root root 2.4M Nov 18 17:57 Cast_Crew_ids.csv
drwx------ 3 root root 4.0K Nov 18 09:04 drive
-rw-r--r-- 1 root root 1.5M Nov 18 10:29 IMDB_5000_movie.csv
-rw-r--r-- 1 root root  93K Nov 18 16:37 Movies_ids.csv
-rw-r--r-- 1 root root 1.5G Nov 18 09:07 movies.zip
drwxr-xr-x 2 root root 4.0K Nov 15 19:33 sample_data
drwxr-xr-x 3 root root 4.0K Nov 18 09:07 themoviedb_data


In [0]:
#We will get an error message. No worries about it!
!mkdir themoviedb_data
!cd themoviedb_data && unzip -q ../people.zip
!ls -hl themoviedb_data

warning [../people.zip]:  76 extra bytes at beginning or within zipfile
  (attempting to process anyway)
error [../people.zip]:  reported length of central directory is
  -76 bytes too long (Atari STZip zipfile?  J.H.Holm ZIPSPLIT 1.1
  zipfile?).  Compensating...
error:  expected central file header signature not found (file #1197557).
  (please check that you have transferred or created the zipfile in the
  appropriate BINARY mode and that you have compiled UnZip properly)
total 45M
drwxr-xr-x 2 root root 12M Oct 28 10:33 movies
drwxr-xr-x 2 root root 33M Oct 28 11:36 people


In [0]:
!ls -1 themoviedb_data/people | wc -l

1197556


##5.02 Checking json files structure.

Similarly as we did with movies, we create a function to help us know the structure of our people files:

In [0]:
def formatCheck(path):
    dfTypeCheck = pd.DataFrame()
    dfAux = pd.DataFrame()
    intI = 0

    for root, dirs, files in os.walk(path):
        for f in files:
            intI += 1
            if f.endswith('.json'):
                fp = os.path.join(root,f)
                with open(fp) as o:
                
                    data = ujson.load(o)
                    dfAux = dfAux.append(pd.DataFrame([(x, 'NotDict', str(type(data[x])), 'aux') for x in data.keys() if not isinstance(data[x], dict)], 
                                                              columns = ['key1', 'key2', 'type', 'aux']))
                    dfAux = dfAux.append(pd.DataFrame([(x, y, str(type(data[x][y])), 'aux') for x in data.keys() if isinstance(data[x], dict) 
                                                   for y in data[x].keys()], columns = ['key1', 'key2', 'type', 'aux']))
                    #Every 1000 files, we group and count (this is more efficient than doing it with every row!)
                    #And print the number of files (just to control how it goes).                    
                    if intI % 1000 == 0:
                        dfTypeCheck = dfTypeCheck.append(dfAux.groupby(['key1','key2','type']).count())
                        dfAux = pd.DataFrame()
                        print(intI)
    #Last grouping:            
    if intI % 1000 != 0:
        dfTypeCheck = dfTypeCheck.append(dfAux.groupby(['key1','key2','type']).count())
        dfAux = pd.DataFrame()

    #And now we group and sum all our counts:
    dfTypeCheck = dfTypeCheck.groupby(['key1','key2','type']).sum()
    
    return dfTypeCheck

In [0]:
#Let's launch the function to get dfTypeCheck with the structure and counts of all our json files.
#Just to see how it goes, we included in formatCheck function a counter every 1000 files.
tic=timeit.default_timer()
path = "./themoviedb_data/people"
dfTypeCheck = pd.DataFrame()
dfTypeCheck = formatCheck(path)
toc=timeit.default_timer()
print((toc - tic)/60, ' min') #elapsed time in minutes

In [0]:
dfTypeCheck

aux
key1                 key2         type                       
adult                NotDict      <class 'bool'>      1197556
also_known_as        NotDict      <class 'list'>      1197556
biography            NotDict      <class 'str'>       1197556
birthday             NotDict      <class 'NoneType'>  1082735
                                  <class 'str'>        114821
deathday             NotDict      <class 'NoneType'>  1166166
                                  <class 'str'>         31390
external_ids         cast         <class 'list'>            2
                     crew         <class 'list'>            2
                     facebook_id  <class 'NoneType'>  1190356
                                  <class 'str'>          7197
                     freebase_id  <class 'NoneType'>  1151955
                                  <class 'str'>         45599
                     freebase_mid <class 'NoneType'>  1125377
                                  <class 'str'>         72177
                     imdb_id      <class 'NoneType'>   635960
                                  <class 'str'>        561591
                     instagram_id <class 'NoneType'>  1190081
                                  <class 'str'>          7473
                     tvrage_id    <class 'NoneType'>  1156373
                                  <class 'int'>         40902
                     twitter_id   <class 'NoneType'>  1186610
                                  <class 'str'>         10943
gender               NotDict      <class 'int'>       1197556
homepage             NotDict      <class 'NoneType'>  1184167
                                  <class 'str'>         13389
id                   NotDict      <class 'int'>       1197556
images               facebook_id  <class 'NoneType'>        2
                     freebase_id  <class 'NoneType'>        1
                                  <class 'str'>             1
                     freebase_mid <class 'NoneType'>        1
                                  <class 'str'>             1
                     imdb_id      <class 'NoneType'>        1
                                  <class 'str'>             1
                     instagram_id <class 'NoneType'>        2
                     profiles     <class 'list'>      1197554
                     tvrage_id    <class 'NoneType'>        1
                                  <class 'int'>             1
                     twitter_id   <class 'NoneType'>        2
imdb_id              NotDict      <class 'str'>       1197553
known_for_department NotDict      <class 'NoneType'>      418
                                  <class 'str'>       1197138
movie_credits        NotDict      <class 'list'>            2
                     cast         <class 'list'>      1197554
                     crew         <class 'list'>      1197554
name                 NotDict      <class 'str'>       1197556
place_of_birth       NotDict      <class 'NoneType'>  1108933
                                  <class 'str'>         88623
popularity           NotDict      <class 'float'>      478769
                                  <class 'int'>        718787
profile_path         NotDict      <class 'NoneType'>  1038845
                                  <class 'str'>        158711
tv_credits           cast         <class 'list'>      1197556
                     crew         <class 'list'>      1197556

In [0]:
!pip install openpyxl
import openpyxl

dfTypeCheck.to_excel('typesCheckPeople.xlsx')

#And then we copy it to our Drive:
!cp typesCheckPeople.xlsx /content/drive/"My Drive"

## 5.03 Importing json files to a pandas dataframe.

After analizing the list of fields in **`dfTypeCheck`** and the documentation of [The Movie Database API](https://developers.themoviedb.org/3/people/get-person-details), we select the fields we will import. Getting the whole information of all fields could take most of the available memory. So this is an aspect to take into account. And this is why we are going to firstly import the csv file with the id's of all people working in our selected universe of movies. At this point, we will only import the information of people for those ids. 
Anyways, with regards to the fields to import, we will take more than we will finally use, as this will allow us to enhance the model we are going to develop in the future.

Thus we create a generator function to import the selected fields. Using a generator function proves to be the most efficient way with regards to memory. While importing the files, we convert the first level of dictionaries into columns of a dataframe, take the fields we find interesting and drop the others. We collect all people information in **`dfPeople`** pandas dataframe.

In [8]:
#Reading file and saving in df
dfPeopleIds = pd.read_csv('./Cast_Crew_ids.csv')
dfPeopleIds.tail()

,Unnamed: 0,id
182312,182312,969564
182313,182313,69450
182314,182314,966377
182315,182315,966378
182316,182316,966376


In [0]:
setPeopleIds = set(dfPeopleIds['id'])

In [0]:
def peopleIterator(path):
    intI = 0
    for root, dirs, files in os.walk(path):
        for f in files:
            intI += 1
            if f.endswith('.json'):
                fp = os.path.join(root,f)
                with open(fp) as o:
                    data = ujson.load(o)
                
                if intI % 10000 == 0:
                    print(intI)
                    
                result = {"adult": data["adult"], 
                          "birthday": data["birthday"], "deathday": data["deathday"], 
                          "gender": data["gender"], "people_id": data["id"], "imdb_id": data.get("imdb_id",None),
                          "known_for_department": data["known_for_department"], "name": data["name"], "place_of_birth": data["place_of_birth"],
                          "popularity": data["popularity"], "tv_credits_cast": data["tv_credits"]["cast"], 
                          "tv_credits_crew": data["tv_credits"]["crew"]}

                #After the dfTypeCheck, we know that movie_credits is not a dictionary in two rows (probably this is an error.). 
                #So we need to take this situation into consideration:
                if isinstance(data["movie_credits"], dict):
                    result["movie_credits_cast"] = data["movie_credits"]["cast"]
                    result["movie_credits_crew"] = data["movie_credits"]["crew"]
                else:
                    result["movie_credits_cast"] = list()
                    result["movie_credits_crew"] = list()

                if data["id"] in setPeopleIds:    
                    yield result                   

10000
20000
30000
40000
50000
60000
70000
80000
90000
100000
110000
120000
130000
140000
150000
160000
170000
180000
190000
200000
210000
220000
230000
240000
250000
260000
270000
280000
290000
300000
310000
320000
330000
340000
350000
360000
370000
380000
390000
400000
410000
420000
430000
440000
450000
460000
470000
480000
490000
500000
510000
520000
530000
540000
550000
560000
570000
580000
590000
600000
610000
620000
630000
640000
650000
660000
670000
680000
690000
700000
710000
720000
730000
740000
750000
760000
770000
780000
790000
800000
810000
820000
830000
840000
850000
860000
870000
880000
890000
900000
910000
920000
930000
940000
950000
960000
970000
980000
990000
1000000
1010000
1020000
1030000
1040000
1050000
1060000
1070000
1080000
1090000
1100000
1110000
1120000
1130000
1140000
1150000
1160000
1170000
1180000
1190000


1030.714719391999

In [0]:
#This script takes about 20min to be executed in Google Colab
path = "./themoviedb_data/people"
dfPeople = pd.DataFrame()
tic=timeit.default_timer()
dfPeople = pd.DataFrame(peopleIterator(path))
toc=timeit.default_timer()
print((toc - tic)/60, ' min') #elapsed time in minutes

In [0]:
dfPeopleIds.shape

(203181, 2)

In [0]:
dfPeople.shape

(203159, 13)

In [0]:
dfPeople.columns

Index(['adult', 'birthday', 'deathday', 'gender', 'imdb_id',
       'known_for_department', 'movie_credits_cast', 'movie_credits_crew',
       'name', 'place_of_birth', 'popularity', 'tv_credits_cast',
       'tv_credits_crew'],
      dtype='object')

Let's confirm **`people_id`** is the primary key of **`dfPeople`**:

In [0]:
dfPeople['people_id'].dtypes

In [0]:
dfPeople['people_id'].isnull().sum()

In [0]:
dfPeople['people_id'].duplicated().sum()

##5.04 Tidying dfPeople dataframe.

As it happened with **`dfMovies`**, there are some columns in **`dfPeople`** made of lists of dictionaries. And again we will define functions to deal with them.

In [0]:
#Firstly we need people ids to be the index of dfPeople (the function below is defined assuming this; this makes it much quicker).
dfPeople.index  = dfPeople['people_id']

In [0]:
def columnIteratorPeople(columnName, *args):
    for id in dfPeople.index:
        for dict in dfPeople[dfPeople.index == id][columnName].iloc[0]:
            outputDict = {'people_id': id}
            for key in args:
                outputDict[key] = dict.get(key,None)
            yield outputDict

In [0]:
def columnProcessesPeople (columnName, *args):
    #I need to declare dfPeople as global as I will change it inside this function.
    global dfPeople
    
    dfOutput = pd.DataFrame()
    dfOutput = pd.DataFrame(columnIteratorPeople(columnName, *args))
    
    #Sanity check:
    dfCheck = pd.DataFrame(dfPeople[columnName].map(len).value_counts())
    dfCheck['calculation']=dfCheck[columnName]*dfCheck.index
    
    if dfOutput.shape[0] == dfCheck['calculation'].sum():
        dfPeople = dfPeople.drop(columnName, axis=1)
        return dfOutput
    else:
        return "Error in process"

###MOVIE CREDITS CAST

In [0]:
columnName = 'movie_credits_cast'

In [0]:
np.sort(dfPeople[columnName].map(len).unique())

array([  0,   1,   2,   3,   4,   5,   6,   7,   8,   9,  10,  11,  12,
        13,  14,  15,  16,  17,  18,  19,  20,  21,  22,  23,  24,  25,
        26,  27,  28,  29,  30,  31,  32,  33,  34,  35,  36,  37,  38,
        39,  40,  41,  42,  43,  44,  45,  46,  47,  48,  49,  50,  51,
        52,  53,  54,  55,  56,  57,  58,  59,  60,  61,  62,  63,  64,
        65,  66,  67,  68,  69,  70,  71,  72,  73,  74,  75,  76,  77,
        78,  79,  80,  81,  82,  83,  84,  85,  86,  87,  88,  89,  90,
        91,  92,  93,  94,  95,  96,  97,  98,  99, 100, 101, 102, 103,
       104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116,
       117, 118, 119, 120, 121, 122, 123, 124, 125, 126, 127, 128, 129,
       130, 131, 132, 133, 134, 135, 136, 137, 138, 139, 140, 141, 142,
       143, 144, 145, 146, 147, 148, 149, 150, 151, 152, 153, 154, 155,
       156, 157, 158, 159, 160, 161, 162, 163, 164, 165, 166, 167, 168,
       169, 171, 172, 174, 175, 176, 177, 178, 179, 180, 181, 18

In [0]:
#Pay attention to the number of 0's!
dfPeople[columnName].map(len).value_counts()

0      90996
1      40328
2      15031
3       9082
4       6376
5       4710
6       3694
7       3048
8       2586
9       2119
10      1926
11      1616
12      1391
13      1313
14      1148
15      1079
16       887
18       834
17       811
19       735
20       687
21       606
22       595
23       549
24       524
25       479
26       469
28       395
29       383
27       377
       ...  
238        1
479        1
290        1
204        1
313        1
160        1
166        1
301        1
540        1
306        1
154        1
180        1
181        1
182        1
184        1
312        1
701        1
201        1
190        1
275        1
192        1
193        1
530        1
321        1
197        1
199        1
271        1
327        1
200        1
183        1
Name: movie_credits_cast, Length: 238, dtype: int64

In [0]:
#Having a look to one example:
dfPeople[dfPeople['people_id']==6885][columnName].iloc[0]

[{'adult': False,
  'backdrop_path': '/pexMctImHaIj8haojBDP5RxhM9X.jpg',
  'character': 'Aileen Wuornos',
  'credit_id': '52fe424bc3a36847f8013009',
  'genre_ids': [80, 18],
  'id': 504,
  'original_language': 'en',
  'original_title': 'Monster',
  'overview': "An emotionally scarred highway hooker shoots a sadistic trick who rapes her, and ultimately becomes America's first female serial killer.",
  'popularity': 7.218,
  'poster_path': '/aevmNtJCNG4ZlfEeEGZ79frMUes.jpg',
  'release_date': '2003-12-24',
  'title': 'Monster',
  'video': False,
  'vote_average': 7.1,
  'vote_count': 752},
 {'adult': False,
  'backdrop_path': '/bTyOP3kqN6a8m9lGh7NCruAnz5p.jpg',
  'character': 'Candy Kendall',
  'credit_id': '52fe430fc3a36847f80376d3',
  'genre_ids': [18],
  'id': 1715,
  'original_language': 'en',
  'original_title': 'The Cider House Rules',
  'overview': 'Homer is an orphan who was never adopted, becoming the favorite of orphanage director Dr. Larch. Dr. Larch imparts his full medical k

In [0]:
dfPeople[dfPeople['people_id']==6885][columnName].iloc[0][0]

{'adult': False,
 'backdrop_path': '/pexMctImHaIj8haojBDP5RxhM9X.jpg',
 'character': 'Aileen Wuornos',
 'credit_id': '52fe424bc3a36847f8013009',
 'genre_ids': [80, 18],
 'id': 504,
 'original_language': 'en',
 'original_title': 'Monster',
 'overview': "An emotionally scarred highway hooker shoots a sadistic trick who rapes her, and ultimately becomes America's first female serial killer.",
 'popularity': 7.218,
 'poster_path': '/aevmNtJCNG4ZlfEeEGZ79frMUes.jpg',
 'release_date': '2003-12-24',
 'title': 'Monster',
 'video': False,
 'vote_average': 7.1,
 'vote_count': 752}

In [0]:
dfPeople[dfPeople['people_id']==6885][columnName].iloc[0][0].keys()

dict_keys(['release_date', 'adult', 'vote_average', 'vote_count', 'video', 'title', 'popularity', 'genre_ids', 'original_language', 'character', 'original_title', 'poster_path', 'id', 'backdrop_path', 'overview', 'credit_id'])

In [0]:
tic=timeit.default_timer()
dfPeopleMoviesCast = columnProcessesPeople (columnName, 'adult', 'character', 'genre_ids', 'id', 'original_language', 'original_title', 'popularity', 'release_date', 'title', 'vote_average', 'vote_count')
toc=timeit.default_timer()
toc - tic #elapsed time in seconds

173.36695612099902

In [0]:
dfPeopleMoviesCast.shape

(996469, 12)

In [0]:
dfPeopleMoviesCast.tail()

In [0]:
#Looking at our example:
dfPeopleMoviesCast[dfPeopleMoviesCast['people_id'] == 6885].iloc[0]

In [0]:
dfPeople.columns

Index(['adult', 'birthday', 'deathday', 'gender', 'imdb_id',
       'known_for_department', 'movie_credits_crew', 'name', 'place_of_birth',
       'popularity', 'tv_credits_cast', 'tv_credits_crew'],
      dtype='object')

In [0]:
dfPeople.shape

####MOVIE CREDITS CREW

In [0]:
columnName = 'movie_credits_crew'

In [0]:
np.sort(dfPeople[columnName].map(len).unique())

array([  0,   1,   2,   3,   4,   5,   6,   7,   8,   9,  10,  11,  12,
        13,  14,  15,  16,  17,  18,  19,  20,  21,  22,  23,  24,  25,
        26,  27,  28,  29,  30,  31,  32,  33,  34,  35,  36,  37,  38,
        39,  40,  41,  42,  43,  44,  45,  46,  47,  48,  49,  50,  51,
        52,  53,  54,  55,  56,  57,  58,  59,  60,  61,  62,  63,  64,
        65,  66,  67,  68,  69,  70,  71,  72,  73,  74,  75,  76,  77,
        78,  79,  80,  81,  82,  83,  84,  85,  86,  87,  88,  89,  90,
        91,  92,  93,  94,  95,  96,  97,  98,  99, 100, 101, 102, 103,
       104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116,
       117, 118, 119, 120, 121, 122, 123, 124, 125, 126, 127, 128, 129,
       131, 132, 133, 134, 135, 136, 137, 138, 139, 140, 141, 142, 144,
       146, 147, 149, 155, 157, 158, 159, 161, 162, 163, 164, 165, 168,
       171, 173, 174, 176, 177, 183, 186, 196, 205, 207, 212, 213, 215,
       217, 218, 219, 229, 231, 234, 236, 248, 252, 256, 257, 26

In [0]:
#Pay attention to the number of 0's!
dfPeople[columnName].map(len).value_counts()

0      89291
1      42005
2      19013
3      11473
4       7699
5       5582
6       4283
7       3316
8       2613
9       2167
10      1792
11      1514
12      1335
13      1073
14      1010
15       865
16       694
17       614
18       604
19       510
20       466
22       377
21       371
23       321
24       277
25       263
26       248
27       218
30       187
28       183
       ...  
309        1
140        1
157        1
620        1
335        1
339        1
155        1
212        1
213        1
215        1
280        1
343        1
217        1
158        1
219        1
334        1
229        1
147        1
173        1
231        1
159        1
234        1
144        1
491        1
108        1
205        1
236        1
397        1
492        1
556        1
Name: movie_credits_crew, Length: 198, dtype: int64

In [0]:
#Looking at one example:
dfPeople[dfPeople['people_id']==6885][columnName].iloc[0]

[{'adult': False,
  'backdrop_path': '/pexMctImHaIj8haojBDP5RxhM9X.jpg',
  'credit_id': '52fe424bc3a36847f8013013',
  'department': 'Production',
  'genre_ids': [80, 18],
  'id': 504,
  'job': 'Producer',
  'original_language': 'en',
  'original_title': 'Monster',
  'overview': "An emotionally scarred highway hooker shoots a sadistic trick who rapes her, and ultimately becomes America's first female serial killer.",
  'popularity': 7.218,
  'poster_path': '/aevmNtJCNG4ZlfEeEGZ79frMUes.jpg',
  'release_date': '2003-12-24',
  'title': 'Monster',
  'video': False,
  'vote_average': 7.1,
  'vote_count': 752},
 {'adult': False,
  'backdrop_path': None,
  'credit_id': '52fe49e69251416c910bc7cf',
  'department': 'Production',
  'genre_ids': [],
  'id': 88021,
  'job': 'Producer',
  'original_language': 'en',
  'original_title': 'East of Havana',
  'overview': 'Documenting the hardships of three gifted Cuban rappers trying to survive in the harsh ghettos of Havana.',
  'popularity': 0.41200000

In [0]:
dfPeople[dfPeople['people_id']==6885][columnName].iloc[0][0]

{'adult': False,
 'backdrop_path': '/pexMctImHaIj8haojBDP5RxhM9X.jpg',
 'credit_id': '52fe424bc3a36847f8013013',
 'department': 'Production',
 'genre_ids': [80, 18],
 'id': 504,
 'job': 'Producer',
 'original_language': 'en',
 'original_title': 'Monster',
 'overview': "An emotionally scarred highway hooker shoots a sadistic trick who rapes her, and ultimately becomes America's first female serial killer.",
 'popularity': 7.218,
 'poster_path': '/aevmNtJCNG4ZlfEeEGZ79frMUes.jpg',
 'release_date': '2003-12-24',
 'title': 'Monster',
 'video': False,
 'vote_average': 7.1,
 'vote_count': 752}

In [0]:
dfPeople[dfPeople['people_id']==6885][columnName].iloc[0][0].keys()

dict_keys(['id', 'department', 'original_language', 'original_title', 'job', 'overview', 'genre_ids', 'video', 'credit_id', 'release_date', 'popularity', 'vote_average', 'vote_count', 'title', 'adult', 'backdrop_path', 'poster_path'])

In [0]:
tic=timeit.default_timer()
dfPeopleMoviesCrew = columnProcessesPeople (columnName, 'adult', 'department', 'genre_ids', 'id', 'job', 'original_language', 'original_title', 'popularity', 'release_date', 'title', 'vote_average', 'vote_count')
toc=timeit.default_timer()
toc - tic #elapsed time in seconds

168.03668097199989

In [0]:
dfPeopleMoviesCrew.shape

(618620, 13)

In [0]:
dfPeopleMoviesCrew.tail()

In [0]:
#Looking at our example:
dfPeopleMoviesCrew[dfPeopleMoviesCrew['people_id'] == 6885]

,adult,department,genre_ids,id,job,original_language,original_title,people_id,popularity,release_date,title,vote_average,vote_count
60806,False,Production,"[80, 18]",504,Producer,en,Monster,6885,7.218,2003-12-24,Monster,7.1,752
60807,False,Production,[],88021,Producer,en,East of Havana,6885,0.412,2006-03-14,East of Havana,6.0,2
60808,False,Production,[35],340021,Producer,en,Everything Nice,6885,0.091,None,Everything Nice,10.0,1
60809,False,Production,"[28, 35, 80]",340022,Producer,en,Gringo,6885,14.822,2018-03-09,Gringo,5.9,190
60810,False,Production,[18],340027,Producer,en,Brain on Fire,6885,6.852,2017-06-06,Brain on Fire,7.2,348
60811,False,Production,"[18, 9648, 53]",182560,Producer,en,Dark Places,6885,8.574,2015-08-07,Dark Places,5.8,638
60812,False,Production,"[28, 9648, 53]",341013,Producer,en,Atomic Blonde,6885,17.113,2017-07-28,Atomic Blonde,6.2,2901
60813,False,Production,"[35, 18]",400579,Producer,en,Tully,6885,19.191,2018-05-04,Tully,6.9,218
60814,False,Production,"[28, 9648, 53]",522958,Producer,en,Atomic Blonde 2,6885,0.675,,Atomic Blonde 2,10.0,1
60815,False,Production,[18],14349,Producer,en,Sleepwalking,6885,3.290,2008-03-14,Sleepwalking,6.4,29


In [0]:
dfPeople.shape

(203159, 11)

In [0]:
dfPeople.columns

Index(['adult', 'birthday', 'deathday', 'gender', 'imdb_id',
       'known_for_department', 'name', 'place_of_birth', 'popularity',
       'tv_credits_cast', 'tv_credits_crew'],
      dtype='object')

###TV CREDITS CAST

In [0]:
columnName = 'tv_credits_cast'

In [0]:
np.sort(dfPeople[columnName].map(len).unique())

array([  0,   1,   2,   3,   4,   5,   6,   7,   8,   9,  10,  11,  12,
        13,  14,  15,  16,  17,  18,  19,  20,  21,  22,  23,  24,  25,
        26,  27,  28,  29,  30,  31,  32,  33,  34,  35,  36,  37,  38,
        39,  40,  41,  42,  43,  44,  45,  46,  47,  48,  49,  50,  51,
        52,  53,  54,  55,  56,  57,  58,  59,  60,  61,  62,  63,  64,
        65,  66,  67,  68,  69,  70,  71,  72,  73,  74,  75,  76,  77,
        78,  79,  80,  81,  82,  83,  84,  85,  86,  87,  88,  89,  90,
        91,  92,  93,  94,  95,  96,  97,  98, 100, 101, 102, 103, 104,
       107, 108, 109, 110, 111, 112, 113, 114, 115, 117, 118, 123, 124,
       130, 140, 224])

In [0]:
#Pay attention to the number of 0's!
dfPeople[columnName].map(len).value_counts()

0      162379
1       11970
2        5450
3        3510
4        2446
5        1870
6        1525
7        1332
8        1097
9         972
10        852
11        808
12        692
13        669
14        580
15        527
16        477
17        450
18        423
19        385
21        370
20        344
23        283
22        276
26        243
24        231
27        212
25        209
28        209
29        189
        ...  
103         3
102         3
117         2
112         2
118         2
111         2
109         2
101         2
87          2
93          2
97          1
110         1
123         1
82          1
130         1
115         1
114         1
113         1
86          1
88          1
91          1
224         1
108         1
107         1
104         1
140         1
94          1
96          1
100         1
124         1
Name: tv_credits_cast, Length: 120, dtype: int64

In [0]:
#Looking at one example:
dfPeople[dfPeople['people_id']==6885][columnName].iloc[0]

[{'backdrop_path': '/7wri8hbMd1yZWIFubnZTe9ZJrKT.jpg',
  'character': '',
  'credit_id': '5253872d19c2957940217c5e',
  'episode_count': 2,
  'first_air_date': '2003-09-08',
  'genre_ids': [35],
  'id': 562,
  'name': 'The Ellen DeGeneres Show',
  'origin_country': ['US'],
  'original_language': 'en',
  'original_name': 'The Ellen DeGeneres Show',
  'overview': 'The Ellen DeGeneres Show, often shortened to Ellen, is an American television talk show hosted by comedian/actress Ellen DeGeneres.',
  'popularity': 5.128,
  'poster_path': '/AvfvsaqYt2oBYDC9c8N926mDHjl.jpg',
  'vote_average': 6.3,
  'vote_count': 35},
 {'backdrop_path': '/lt0GdLpUgFI09hkbTgvx8MvRS9m.jpg',
  'character': '',
  'credit_id': '525707e3760ee3776a0395b5',
  'episode_count': 3,
  'first_air_date': '2003-01-26',
  'genre_ids': [35, 10767],
  'id': 1489,
  'name': 'Jimmy Kimmel Live!',
  'origin_country': ['US'],
  'original_language': 'en',
  'original_name': 'Jimmy Kimmel Live!',
  'overview': 'Jimmy Kimmel Live! is 

In [0]:
dfPeople[dfPeople['people_id']==6885][columnName].iloc[0][0]

{'backdrop_path': '/7wri8hbMd1yZWIFubnZTe9ZJrKT.jpg',
 'character': '',
 'credit_id': '5253872d19c2957940217c5e',
 'episode_count': 2,
 'first_air_date': '2003-09-08',
 'genre_ids': [35],
 'id': 562,
 'name': 'The Ellen DeGeneres Show',
 'origin_country': ['US'],
 'original_language': 'en',
 'original_name': 'The Ellen DeGeneres Show',
 'overview': 'The Ellen DeGeneres Show, often shortened to Ellen, is an American television talk show hosted by comedian/actress Ellen DeGeneres.',
 'popularity': 5.128,
 'poster_path': '/AvfvsaqYt2oBYDC9c8N926mDHjl.jpg',
 'vote_average': 6.3,
 'vote_count': 35}

In [0]:
dfPeople[dfPeople.index ==6885][columnName].iloc[0][0].keys()

dict_keys(['origin_country', 'original_name', 'genre_ids', 'vote_count', 'backdrop_path', 'name', 'first_air_date', 'original_language', 'popularity', 'character', 'episode_count', 'id', 'credit_id', 'vote_average', 'overview', 'poster_path'])

In [0]:
tic=timeit.default_timer()
dfPeopleTvCast = columnProcessesPeople (columnName, 'character', 'episode_count', 'first_air_date', 'genre_ids', 'id', 'name', 'origin_country', 'original_language', 'original_name', 'popularity', 'vote_average', 'vote_count')
toc=timeit.default_timer()
toc - tic #elapsed time in seconds

160.8628852460024

In [0]:
dfPeopleTvCast.shape

(325921, 13)

In [0]:
dfPeopleTvCast.tail()

,character,episode_count,first_air_date,genre_ids,id,name,origin_country,original_language,original_name,people_id,popularity,vote_average,vote_count
325916,"Penelope ""Penny"" Halliwell",15.0,1998-10-07,"[35, 18, 9648]",1981,Charmed,[US],en,Charmed,27568,16.368,6.8,240
325917,Bus Driver,1.0,2016-09-21,[35],67040,Speechless,[US],en,Speechless,165359,9.482,7.5,44
325918,,1.0,1980-12-11,"[18, 10759]",734,"Magnum, P.I.",[US],en,"Magnum, P.I.",106811,25.180,7.1,75
325919,,1.0,1973-10-24,"[18, 10759]",3572,Kojak,[US],en,Kojak,106811,2.597,7.3,31
325920,,1.0,1972-09-13,"[10759, 10765]",11398,Search,[US],en,Search,106811,1.317,8.9,4


In [0]:
#Looking at our example:
dfPeopleTvCast[dfPeopleTvCast['people_id'] == 6885]

,character,episode_count,first_air_date,genre_ids,id,name,origin_country,original_language,original_name,people_id,popularity,vote_average,vote_count
32136,,2.0,2003-09-08,[35],562,The Ellen DeGeneres Show,[US],en,The Ellen DeGeneres Show,6885,5.128,6.3,35
32137,,3.0,2003-01-26,"[35, 10767]",1489,Jimmy Kimmel Live!,[US],en,Jimmy Kimmel Live!,6885,2.836,5.9,34
32138,,3.0,1983-04-04,[],1900,Live with Regis and Kathie Lee,[US],en,Live with Regis and Kathie Lee,6885,2.536,4.9,4
32139,,1.0,1997-08-11,[10767],2221,The View,[US],en,The View,6885,17.604,3.7,23
32140,,2.0,1996-07-22,"[35, 10763]",2224,The Daily Show with Trevor Noah,[US],en,The Daily Show with Trevor Noah,6885,43.025,6.7,195
32141,,1.0,1992-05-25,"[35, 10767]",2518,The Tonight Show with Jay Leno,[US],en,The Tonight Show with Jay Leno,6885,14.895,5.0,35
32142,,3.0,1993-09-13,"[35, 10767]",4573,Late Night with Conan O'Brien,[US],en,Late Night with Conan O'Brien,6885,17.536,7.4,52
32143,,5.0,2003-11-02,[35],4589,Arrested Development,[US],en,Arrested Development,6885,14.924,8.2,418
32144,Rita,26.0,2003-11-02,[35],4589,Arrested Development,[US],en,Arrested Development,6885,14.924,8.2,418
32145,,1.0,1994-06-01,[10767],217,Inside the Actors Studio,[],en,Inside the Actors Studio,6885,0.459,7.6,20


In [0]:
dfPeople.shape

(203159, 10)

In [0]:
dfPeople.columns

Index(['adult', 'birthday', 'deathday', 'gender', 'imdb_id',
       'known_for_department', 'name', 'place_of_birth', 'popularity',
       'tv_credits_crew'],
      dtype='object')

### TV CREDITS CREW

In [0]:
columnName = 'tv_credits_crew'

In [0]:
np.sort(dfPeople[columnName].map(len).unique())

array([  0,   1,   2,   3,   4,   5,   6,   7,   8,   9,  10,  11,  12,
        13,  14,  15,  16,  17,  18,  19,  20,  21,  22,  23,  24,  25,
        26,  27,  28,  29,  30,  31,  32,  33,  34,  35,  36,  37,  38,
        39,  40,  41,  42,  43,  44,  45,  46,  47,  48,  49,  50,  51,
        52,  53,  55,  59,  60,  63,  64,  70,  89, 131, 132])

In [0]:
#Pay attention to the number of 0's!
dfPeople[columnName].map(len).value_counts()

0      190368
1        7280
2        2109
3         980
4         503
5         355
6         234
7         208
8         147
9         114
10        112
11        100
12         68
13         56
14         53
15         44
17         37
18         32
16         31
22         22
23         22
19         21
25         21
20         20
21         18
30         14
24         14
32         14
26         13
29         12
        ...  
27         10
35          9
34          8
37          8
31          7
46          6
42          5
43          5
44          4
39          4
41          3
64          3
52          3
36          3
53          3
63          2
60          2
59          2
45          2
50          2
49          2
48          2
51          1
55          1
132         1
131         1
47          1
70          1
40          1
89          1
Name: tv_credits_crew, Length: 63, dtype: int64

In [0]:
#Looking at one example:
dfPeople[dfPeople['people_id']==6885][columnName].iloc[0]

[{'backdrop_path': '/1Dxe2ZXesgLsbpJI5m6PyKIVk93.jpg',
  'credit_id': '56e1f15e9251417896000357',
  'department': 'Production',
  'episode_count': 13,
  'first_air_date': '2017-04-21',
  'genre_ids': [35],
  'id': 65856,
  'job': 'Executive Producer',
  'name': 'Girlboss',
  'origin_country': ['US'],
  'original_language': 'en',
  'original_name': 'Girlboss',
  'overview': 'Sophia is a rebellious, broke anarchist who refuses to grow up. She stumbles upon her passion of selling vintage clothes online and becomes an unlikely businesswoman. As she builds her retail fashion empire, she realizes the value and the difficulty of being the boss of her own life.',
  'popularity': 5.702,
  'poster_path': '/w60XVdh93UU9k7yyXFItnZkVA5y.jpg',
  'vote_average': 6.8,
  'vote_count': 33},
 {'backdrop_path': '/a906PH7CDmSOdS7kmnAgdWk5mhv.jpg',
  'credit_id': '58ba5a70925141607b019714',
  'department': 'Production',
  'episode_count': 10,
  'first_air_date': '2017-10-13',
  'genre_ids': [80, 18],
  'id'

In [0]:
dfPeople[dfPeople.index ==6885][columnName].iloc[0][0]

{'backdrop_path': '/1Dxe2ZXesgLsbpJI5m6PyKIVk93.jpg',
 'credit_id': '56e1f15e9251417896000357',
 'department': 'Production',
 'episode_count': 13,
 'first_air_date': '2017-04-21',
 'genre_ids': [35],
 'id': 65856,
 'job': 'Executive Producer',
 'name': 'Girlboss',
 'origin_country': ['US'],
 'original_language': 'en',
 'original_name': 'Girlboss',
 'overview': 'Sophia is a rebellious, broke anarchist who refuses to grow up. She stumbles upon her passion of selling vintage clothes online and becomes an unlikely businesswoman. As she builds her retail fashion empire, she realizes the value and the difficulty of being the boss of her own life.',
 'popularity': 5.702,
 'poster_path': '/w60XVdh93UU9k7yyXFItnZkVA5y.jpg',
 'vote_average': 6.8,
 'vote_count': 33}

In [0]:
dfPeople[dfPeople.index ==6885][columnName].iloc[0][0].keys()

dict_keys(['id', 'department', 'original_language', 'episode_count', 'job', 'overview', 'origin_country', 'original_name', 'vote_count', 'name', 'popularity', 'credit_id', 'backdrop_path', 'first_air_date', 'vote_average', 'genre_ids', 'poster_path'])

In [0]:
tic=timeit.default_timer()
dfPeopleTvCrew = columnProcessesPeople (columnName, 'department', 'episode_count', 'first_air_date', 'genre_ids', 'id', 'job', 'name', 'origin_country', 'original_language', 'original_name', 'popularity', 'vote_average', 'vote_count')
toc=timeit.default_timer()
toc - tic #elapsed time in seconds

157.40268017500057

In [0]:
dfPeopleTvCrew.shape

(40362, 14)

In [0]:
dfPeopleTvCrew.tail()

,department,episode_count,first_air_date,genre_ids,id,job,name,origin_country,original_language,original_name,people_id,popularity,vote_average,vote_count
40357,Directing,12,2014-04-11,"[16, 10759, 10765]",60831,Director,Knights of Sidonia,[JP],ja,シドニアの騎士,237072,3.529,7.7,20
40358,Editing,2,2016-09-20,"[35, 18]",67136,Assistant Editor,This Is Us,[US],en,This Is Us,1730317,24.050,8.1,264
40359,Costume & Make-Up,1,2011-04-17,"[18, 10759, 10765]",1399,Makeup Effects Designer,Game of Thrones,[US],en,Game of Thrones,101608,56.630,8.2,4792
40360,Production,6,2017-04-04,[10765],70690,Casting,Dimension 404,[US],en,Dimension 404,1021206,5.911,6.9,22
40361,Crew,1,2009-09-22,[18],1435,Transportation Coordinator,The Good Wife,[US],en,The Good Wife,1410317,26.867,7.4,170


In [0]:
#Looking at our example:
dfPeopleTvCrew[dfPeopleTvCrew['people_id'] == 6885]

,department,episode_count,first_air_date,genre_ids,id,job,name,origin_country,original_language,original_name,people_id,popularity,vote_average,vote_count
4028,Production,13,2017-04-21,[35],65856,Executive Producer,Girlboss,[US],en,Girlboss,6885,5.702,6.8,33
4029,Production,10,2017-10-13,"[80, 18]",67744,Executive Producer,Mindhunter,[US],en,Mindhunter,6885,45.600,7.4,443
4030,Production,1,2019-01-01,[18],81508,Producer,A Life in Men,[],en,A Life in Men,6885,0.008,0.0,0


In [0]:
dfPeople.shape

(203159, 9)

In [0]:
dfPeople.columns

Index(['adult', 'birthday', 'deathday', 'gender', 'imdb_id',
       'known_for_department', 'name', 'place_of_birth', 'popularity'],
      dtype='object')